# Exporting data
To export data from a forecast to a csv or Excel file, you can use the `/operations/export` ("Export") endpoint.

**Note**: You can fetch similar data from the `forecast-contents/{forecastId}/variables` ("Variables") endpoint. Here are the key differences between the two endpoints.

&nbsp;|Variables|Export
:--|:--|:--
Suitable for|Small selections only|Selections of any size
How does the data arrive?|JSON array in response|A background operation is triggered to generate a csv, to be downloaded (see below)
Supports transformations and aggregations?|No|Yes
Supports date range?|No|Yes

## Initial setup
- Import libraries
- Specify the GMWO API base URL and your access token

In [1]:
import requests
import urllib.request
import pandas # Assumes pandas has been installed already

base_url = 'https://model.oxfordeconomics.com/api'
access_token = <See https://model.oxfordeconomics.com/api/docs/#authentication>
headers = {'Authorization' : f'Bearer {access_token}'}
releasesFolder = "oxford-economics/releases/gem"

    

## Find a source forecast
You can export data from a single forecast at a time. You'll need to know the full path to the forecast in GMWO. E.g. `"/oxford-economics/releases/GEM/Oct23_1 25yr"`.

In this example, we use the `/resources` endpoint to find the path to the latest GEM release.

In [2]:
gemFolderResponse = requests.get(f"{base_url}/v1/resources/{releasesFolder}", headers=headers)
gemFolder = gemFolderResponse.json()

# Get forecasts only (i.e. filter out subdirectories and other file types)
gemForecasts = filter(lambda item: (item["Type"] == 'Forecast'), gemFolder["Children"])

# Sort by latest version of each forecast
gemForecastsLatestFirst = sorted(gemForecasts, key=lambda forecast: (forecast["Versions"][-1]["CreatedAt"]), reverse=True)
sourceForecast = gemForecastsLatestFirst[0]
sourceForecast

{'Type': 'Forecast',
 'EconomicDomain': 'MACRO',
 'ForecastUrl': '/v1/forecasts/=2198fad7-c646-43f7-b81c-f86dd4eea113',
 'Versions': [{'Name': 'Oct23_1 25yr',
   'EconomicDomain': 'MACRO',
   'Range': {'From': '1980Q1', 'To': '2050Q4'},
   'Id': 'd3203eab-fb65-41dc-83d9-1fb87e1c3158',
   'Version': 0,
   'CreatedAt': '2023-10-11T18:55:49.3903852+00:00'}],
 'ResourceType': 'Forecast',
 'Id': '2198fad7-c646-43f7-b81c-f86dd4eea113',
 'Name': 'Oct23_1 25yr',
 'Path': '/oxford-economics/releases/GEM/Oct23_1 25yr',
 'Archiving': None,
 'Product': {'TypeCode': 'OEF', 'Code': 'GBLMACRLM25_ONLINE'}}

## Trigger the export
Send a request to initiate the export. The response will be a "Queued" operation. You must send further requests to wait for this operation to complete.

**Note**: requests to enqueue an operation such as this are subject to stricter rate limits. Please refer to the [`Throttling`](https://model.oxfordeconomics.com/api/docs/#throttling-request-limits-) section in the API guide.

In [5]:
# The export request body expects JSON of the form below.
# Optional properties are commented out, with their default values shown

# You can build the series selection in any way you like.
# In this example, we're making a cross-product of indicator and location lists
indicators = ["CPI", "C"]
locations = ["UK", "GERMANY"]
seriesSelection =  [ { 
        "Indicator": indicator, 
        "Location": location,

        "Transformation": "L"

        ## Transformation options are:
        ## - "L" = level values
        ## - "PY" = percentange change, year-on-year
        ## - "DY" = difference, year-on-year
        ## - "GR" = annualized percentage change
        ## - "P" = percentange change, quarter-on-quarter
        ## - "D" = difference, quarter-on-quarter

        ## Type of values to export ("Variable" or "Residual")
        # "ValueType": "Variable"
    } 
    for indicator in indicators for location in locations
]

exportRequest = {
    # Path to the source forecast. If you already know this, then replace with e.g. "oxford-economics/releases/GEM/Oct23_1 25yr"
    "InputForecast": sourceForecast["Path"], 

    # Range of periods for which to export data
    "Range": {
        "From": "2020Q1",
        "To": "2023Q4"
    },

    # List of series to export
    "SeriesSelection": seriesSelection,

    ## Output file format
    # "Format": "Default"

    ## Format options are:
    ## - Default. One row per series. Various metadata columns. Includes headers.
    ## - DefaultExtended. One row per series. A few extra metadata columns compared to Default. Includes headers.
    ## - Classic_v. Similar to Model software's one-column-per-series export.
    ## - Classic_h. Similar to Model software's Vars By Row export.
    ## - Skinny. Tall narrow format for import into cubes.
    ## - DatabankCompatible. One row per series, matching format of Global Data Workstation, Excel Data Workstation.
    ## - DatabankCompatibleStacked. One row per series and period, matching format of Global Data Workstation, Excel Data Workstation.

    ## Enable/disable aggregation of quarterly data to annual (True or False)
    # "AnnualRollup": False
    
}
exportOperationResponse = requests.post(f"{base_url}/v1/operations/export", headers=headers, json=exportRequest)
exportOperation = exportOperationResponse.json()
exportOperation


{'Artifacts': [],
 'Resources': [],
 'Id': '23614',
 'CreatedAt': '2023-10-16T10:30:14.22+00:00',
 'StartedAt': None,
 'CompletedAt': None,
 'Status': 'Queued',
 'Duration': None,
 'FailureReason': None,
 'Name': None}

## Wait for the export operation to complete
The operation's `await` endpoint is designed to respond as soon as the operation has finished (within 1 minute). You can continue sending `await` requests until the status of the returned operation is no longer in progress, in case the operation takes longer than 1 minute to complete.

In [6]:
operationId = exportOperation["Id"]
while exportOperation["Status"] in ["Queued", "InProgress"]:
    exportOperationResponse = requests.get(f"{base_url}/v1/operations/{operationId}/await", headers=headers)
    exportOperation = exportOperationResponse.json()
    
exportOperation

{'Artifacts': [{'Id': '411cece1-104f-415e-aac9-1f7d740dac26',
   'Filename': 'series-2020-2023_Oct23_1 25yr.csv',
   'Type': 'text/csv',
   'DownloadUrl': 'https://model.oxfordeconomics.com/api/v1/operations/23614/artifact/411cece1-104f-415e-aac9-1f7d740dac26'}],
 'Resources': [{'Id': '2198fad7-c646-43f7-b81c-f86dd4eea113',
   'Path': '/oxford-economics/releases/GEM/Oct23_1 25yr',
   'Version': 0,
   'Role': 'Input'}],
 'Id': '23614',
 'CreatedAt': '2023-10-16T10:30:14.22+00:00',
 'StartedAt': '2023-10-16T10:30:14.3294088+00:00',
 'CompletedAt': '2023-10-16T10:30:49.4809965+00:00',
 'Status': 'Succeeded',
 'Duration': 35151,
 'FailureReason': None,
 'Name': None}

## Download the generated csv file
The generated file is the first (and only) "artifact" of the operation. Use its `DownloadUrl` to download the file contents. 

The `DownloadGmwoResponse()` helper function defined here achieves two things:
- supplies the `Authorization` header to authenticate the download request
- downloads the response to a file without holding the entire file contents in memory at any one time

In [7]:
# Downloads the response from a URL into a file, providing the access token via the Authorization header
def DownloadGmwoResponse(url, filepath):
    opener = urllib.request.build_opener()
    opener.addheaders = [('Authorization', f'Bearer {access_token}')]
    urllib.request.install_opener(opener)
    urllib.request.urlretrieve(url, filepath)
    
exportFile = exportOperation["Artifacts"][0]
exportFilename = exportFile["Filename"]
DownloadGmwoResponse(exportFile["DownloadUrl"], exportFilename)

exportFilename # This file has been downloaded to the same location as this Python notebook

'series-2020-2023_Oct23_1 25yr.csv'

## Use the csv file
You can use the csv file in any number of ways. In this example, we read the data into a pandas dataframe.

In [8]:
data = pandas.read_csv(exportFilename)
data

,Location,Indicator,Units,Scale,Measurement,Frequency,202001,202002,202003,202004,...,202303,202304,Source,Seasonally adjusted,HistoricalEndYear,HistoricalEndQuarter,Source details,Additional source details,Location code,Indicator code
0,United Kingdom,Consumer price index,NaN,2015=100,LevelValues,Quarterly,108.4667,108.5333,108.9333,109.0667,...,131.4356,132.385600,Office for National Statistics\Haver Analytics,No,2023,2,"UK: Consumer Price Index: All Items (NSA, 2015...",D7BT@UK,UK,CPI
1,Germany,Consumer price index,NaN,2020=100,LevelValues,Quarterly,100.2667,100.2333,99.6000,99.9000,...,117.3000,117.873886,Deutsche Bundesbank\Haver Analytics,Yes,2023,3,"Germany: Consumer Price Index (SWDA, 2020=100)",DESPCT@GERMANY,GERMANY,CPI
2,United Kingdom,"Consumption, private, real, LCU",Pound,Millions: chained 2019 Prices,LevelValues,Quarterly,334308.0000,256744.0000,307192.0000,301652.0000,...,341371.4000,341763.220000,Office for National Statistics\Haver Analytics,Yes,2023,2,U.K.: Household Final Consumption Expenditure ...,ABJRQ@UK,UK,C
3,Germany,"Consumption, private, real, LCU",Euro,Billions: chained 2015 Prices,LevelValues,Quarterly,420.0700,374.9000,414.0600,406.0000,...,423.7834,425.478670,Federal Statistics Office\Haver Analytics,Yes,2023,2,"Germany: GDP: Private Consumption (SWDA, Bil.C...",DESNCVC@GERMANY,GERMANY,C
